In [1]:
pip install rdflib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 8.7 MB/s eta 0:00:00


In [2]:
from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal, URIRef

g = Graph()

# Пространство имён
EX = Namespace("http://example.org/ontology/")
g.bind("ex", EX)

# Создаём классы
g.add((EX.Person, RDF.type, OWL.Class))
g.add((EX.Organization, RDF.type, OWL.Class))
g.add((EX.Event, RDF.type, OWL.Class))

# Свойства
g.add((EX.works_for, RDF.type, OWL.ObjectProperty))
g.add((EX.participated_in, RDF.type, OWL.ObjectProperty))

# Индивиды
g.add((EX.ElonMusk, RDF.type, EX.Person))
g.add((EX.SpaceX, RDF.type, EX.Organization))
g.add((EX.Launch2023, RDF.type, EX.Event))

# Логические тройки (knowledge base population)
g.add((EX.ElonMusk, EX.works_for, EX.SpaceX))
g.add((EX.ElonMusk, EX.participated_in, EX.Launch2023))

# Экспорт
g.serialize("ontology.rdf", format="xml")
print("Ontology saved!")


Ontology saved!


In [4]:
print(g)

[a rdfg:Graph;rdflib:storage [a rdflib:Store;rdfs:label 'Memory']].


In [3]:


from transformers import pipeline

ner = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

text = "Elon Musk founded SpaceX in California in 2002."

entities = ner(text)
entities


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


[{'entity_group': 'ORG',
  'score': np.float32(0.66747177),
  'word': 'Elon',
  'start': 0,
  'end': 4},
 {'entity_group': 'PER',
  'score': np.float32(0.83128965),
  'word': 'Mu',
  'start': 5,
  'end': 7},
 {'entity_group': 'ORG',
  'score': np.float32(0.54482806),
  'word': '##sk',
  'start': 7,
  'end': 9},
 {'entity_group': 'ORG',
  'score': np.float32(0.9992529),
  'word': 'SpaceX',
  'start': 18,
  'end': 24},
 {'entity_group': 'LOC',
  'score': np.float32(0.9996731),
  'word': 'California',
  'start': 28,
  'end': 38}]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "Jean-Baptiste/ roberta-large-ner-english"
# примерная модель, можешь заменить на собственную

tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSequenceClassification.from_pretrained("Babelscape/rebel-large")

def extract_relation(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()
    return pred


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at Babelscape/rebel-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

def extract_triplets(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=256)
    raw = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return raw

text = "Elon Musk founded SpaceX in 2002 in California."
print(extract_triplets(text))


 Elon Musk  SpaceX  owner of  SpaceX  Elon Musk  founded by  2002  inception


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

def make_triplets(text):
    prompt = "Extract knowledge graph triplets from the text: " + text
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=128)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

text = "Илон Маск основал компанию SpaceX в 2002 году в Калифорнии."
make_triplets(text)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

True


In [8]:
!pip install neo4j


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 6.3 MB/s eta 0:00:00


In [10]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

def add_triplet(tx, s, r, o):
    tx.run("""
        MERGE (a:Entity {name: $s})
        MERGE (b:Entity {name: $o})
        MERGE (a)-[rel:RELATION {type: $r}]->(b)
    """, s=s, r=r, o=o)

triplets = [
    ("Elon Musk", "founded", "SpaceX"),
    ("SpaceX", "founded_in", "2002"),
    ("SpaceX", "located_in", "California"),
]

with driver.session() as session:
    for s, r, o in triplets:
        session.write_transaction(add_triplet, s, r, o)

print("✔ Triplets saved to Neo4j!")


AttributeError: 'Session' object has no attribute 'write_transaction'

In [11]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

def add_triplet(tx, s, r, o):
    tx.run("""
        MERGE (a:Entity {name: $s})
        MERGE (b:Entity {name: $o})
        MERGE (a)-[rel:RELATION {type: $r}]->(b)
    """, s=s, r=r, o=o)


triplets = [
    ("Elon Musk", "founded", "SpaceX"),
    ("SpaceX", "founded_in", "2002"),
    ("SpaceX", "located_in", "California"),
]

with driver.session() as session:
    for s, r, o in triplets:
        session.execute_write(add_triplet, s, r, o)

print("✔ Triplets saved to Neo4j!")


Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused))
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (

ServiceUnavailable: Couldn't connect to localhost:7687 (resolved to ('[::1]:7687', '127.0.0.1:7687')):
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)